In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import random
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scl-2021-ds/sampleSubmission.csv
/kaggle/input/scl-2021-ds/train.csv
/kaggle/input/scl-2021-ds/test.csv


In [11]:
df_sub = pd.read_csv("../input/scl-2021-ds/test.csv")
submission = pd.DataFrame({
    'id':df_sub['id'],
    'POI/street':'/'
    })
submission.to_csv('7PERCENT.csv', header=True, index=False)

In [12]:
df = pd.read_csv('/kaggle/input/scl-2021-ds/train.csv')

In [13]:
df['POI'] = df['POI/street'].str.split("/", n = 2, expand = True)[0]
df['street'] = df['POI/street'].str.split("/", n = 2, expand = True)[1]

In [14]:
df.replace("", np.nan).head(50)

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,NaN,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,NaN,NaN
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,NaN,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,NaN
4,4,jl. orde baru,/jl. orde baru,NaN,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra...",/kem mel raya,NaN,kem mel raya
7,7,tela keuramat kuta alam,/tela,NaN,tela
8,8,gg. i wates magersari,/gg. i,NaN,gg. i
9,9,bunga ncole ix 2,/bunga ncole ix,NaN,bunga ncole ix


In [15]:
def getIndex(source, txt, label):
    start = source.find(txt)
    if start == -1:
        return start
    end = start + len(txt)
    return (start, end, label)

In [16]:
train_data = []
for index, row in df.iterrows():
    li = []
    di = {}
    for x in ['POI', 'street']:
        if row[x]:
            a = getIndex(row['raw_address'], row[x], x)
            if a != -1:
                li.append(a)
    di['entities'] = li
    train_data.append((row['raw_address'], di))

In [17]:
train_data

[('jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat',
  {'entities': [(0, 40, 'street')]}),
 ('aye, jati sampurna', {'entities': []}),
 ('setu siung 119 rt 5 1 13880 cipayung', {'entities': [(5, 10, 'street')]}),
 ('toko dita, kertosono', {'entities': [(0, 9, 'POI')]}),
 ('jl. orde baru', {'entities': [(0, 13, 'street')]}),
 ('raya samb gede, 299 toko bb kids',
  {'entities': [(20, 32, 'POI'), (0, 14, 'street')]}),
 ('kem mel raya, no 4 bojong rawalumbu rt 1 36 rawalumbu',
  {'entities': [(0, 12, 'street')]}),
 ('tela keuramat kuta alam', {'entities': [(0, 4, 'street')]}),
 ('gg. i wates magersari', {'entities': [(0, 5, 'street')]}),
 ('bunga ncole ix 2', {'entities': [(0, 14, 'street')]}),
 ('cikahuripan sd neg boj 02 klap boj, no 5 16877',
  {'entities': [(26, 34, 'street')]}),
 ('yaya atohar,', {'entities': []}),
 ('abim ix 24 5', {'entities': [(0, 7, 'street')]}),
 ('gang xiii rungkut', {'entities': [(0, 9, 'street')]}),
 ('kamp utan jaya, 23 rt 3 rw 8 16433 beji'

In [18]:
from spacy.tokens import Span

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.blank('id')

In [19]:
# doc = nlp("fb is hiring a new vice president of global policy")
# ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
# print('Before', ents)
# # The model didn't recognize "fb" as an entity :(

# fb_ent = Span(doc, 0, 1, label="ORG") # create a Span for the new entity
# doc.ents = list(doc.ents) + [fb_ent]

# ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
# print('After', ents)

In [20]:
def train_model(train_data):
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    
    for _, annotation in tqdm(train_data):
        for ent in annotation['entities']:
            ner.add_label(ent[2])
            
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in tqdm(range(10)):
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update([text], [annotations], drop=0.2, sgd=optimizer, losses=losses)
                except Exception as e:
                    pass

In [ ]:
train_model(train_data)

100%|██████████| 300000/300000 [00:01<00:00, 258715.48it/s]
/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  **kwargs
/opt/conda/lib/python3.7/site-packages/spacy/language.py:639: UserWarning: [W034] Please install the package spacy-lookups-data in order to include the default lexeme normalization table for the language 'id'.
  **kwargs
/opt/conda/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "tambaksari tam 51 rt 2 rw 11 60136 tambaksari" with entities "[(0, 3, 'street')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold

In [ ]:
testdf = pd.read_csv("../input/scl-2021-ds/test.csv")

In [ ]:
nlp.to_disk('nlp_model')
nlp_model = spacy.load('nlp_model')

In [ ]:
testdf.head()

In [ ]:
out = []

for _, row in tqdm(testdf.iterrows()):
    doc = nlp_model(row.raw_address)
    text = ''
    for ent in doc.ents:
        if ent.label_ == 'POI':
            text += ent.text
            text += '/'
        if ent.label_ == 'street':
            text += '/'
            text += ent.text
    if text:
        out.append(text.replace("//", "/"))
    else:
        out.append('/')

In [ ]:
submission = pd.DataFrame({'id':testdf['id'], 'POI/street': out})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('7PERCENT.csv', header=True, index=False)

In [ ]:
for ent in doc.ents:
        print(f'{ent.label_.upper():{30}}- {ent.text}')